In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

### Gemini 3 and Gemini 2.5 Response Format 

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage

gemini3 = "gemini-3-pro-preview"
gemini2 = "gemini-2.5-pro"

system_msg = SystemMessage("You are a helpful assistant.")

query = "Explain the theory of relativity in simple terms."
messages = [system_msg, HumanMessage(query)]


In [ ]:
model = ChatGoogleGenerativeAI(model=gemini3)
response = model.invoke(messages)

In [ ]:
response

In [ ]:
response.text

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)
response = model.invoke(messages)

In [ ]:
response

In [ ]:
response.content
response.content_blocks
response.usage_metadata
response.response_metadata


### Multimodal usage

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini3)

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the image provided."},
        {
            "type": "image",
            "url": "https://www.shutterstock.com/image-vector/vector-cute-baby-panda-cartoon-600nw-2427356853.jpg",
        },
    ]
)
response = model.invoke([message])

In [ ]:
response.content
response.text

In [ ]:
import base64

image_bytes = open("data/images/panda.png", "rb").read()
bytes_base64 = base64.b64encode(image_bytes).decode("utf-8")

mime_type = "image/png"

## pdf mime type example
# mime_type = "application/pdf", type = "file"

## audio mime type example
# mime_type = "audio/mpeg", type = "audio"

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the image provided."},
        {
            "type": "image",
            "base64": bytes_base64,
            "mime_type": mime_type,
        },
    ]
)
response = model.invoke([message])

In [ ]:
response

### Tool Calling

In [ ]:
# first show ollama web_search tool
# then build weather tool

from scripts import base_tools

In [ ]:
response = base_tools.web_search.invoke({'query': 'what is the latest US stock market updates?'})
# response

In [ ]:
response = base_tools.get_weather.invoke({'location': 'Mumbai'})
# response

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)
model_with_tools = model.bind_tools([base_tools.web_search, base_tools.get_weather])

In [ ]:
model_with_tools
response = model_with_tools.invoke("what is the weather in Mumbai today? and latest news on US stock market?")

In [ ]:
response

### Thinking support

https://ai.google.dev/gemini-api/docs/thinking

With reasoning models, you have the option to adjust the number of internal thinking tokens used (thinking_budget) or to disable thinking altogether.


In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2,
                               thinking_budget=1024,
                               include_thoughts=True)

query = "explain the theory of relativity in simple terms."
response = model.invoke(query)

In [ ]:
print(response)
# print(response.text)

response.content_blocks

response

In [ ]:
model_with_tools = model.bind_tools([base_tools.web_search, base_tools.get_weather])

response = model_with_tools.invoke(query)


In [ ]:
response.content_blocks
response

### Built-in tools

Google Gemini supports a variety of built-in tools, which can be bound to the model in the usual way.

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)

#  base_tools.get_weather -> function calling with tool use is unsupported.
model_with_tools = model.bind_tools([{"google_search": {}}, {"code_execution": {}}])

In [ ]:
query = "When is the next total solar eclipse in the US and what is 3 + 2?"
response = model_with_tools.invoke(query)

In [ ]:
print(response.text)

In [ ]:
response.response_metadata

### Structured Output

In [ ]:
# Use weather tool for the sample structred output
# fields -> location:str, date:str, temperature:str, condition:str

from pydantic import BaseModel

class WeatherOutput(BaseModel):
    location: str
    date: str
    temperature: str
    condition: str

model = ChatGoogleGenerativeAI(model=gemini2)
model_with_tools = model.bind_tools([base_tools.get_weather])

structured_model = model_with_tools.with_structured_output(WeatherOutput)

In [ ]:

response = structured_model.invoke("what is the weather in Mumbai today?")

In [ ]:
response